In [37]:
import numpy as np
import pandas as pd

from typing import Optional
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast, GPT2LMHeadModel
import fastai
from fastai.text.all import *
import re

ModuleNotFoundError: No module named 'fastai'

## 노래 데이터

In [2]:
lyrics1 = pd.read_csv("./datasets/label_result_song_short.csv")
lyrics1.head()

,index,id,title,singer,genres,lyrics,sentiment,sentiment_label
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,2,분노
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,5,당황
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,0,기쁨
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,0,기쁨
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,2,분노


In [4]:
lyrics2 = pd.read_csv("./datasets/translated_lyrics.csv")
lyrics2.head()

,index,id,title,singer,genres,lyrics,translated_lyrics
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,Ay ay ay ay ay
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,거울 속에 내 표정 봐 봐
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,느낌 그래서 좋아요. 기다려온 D-하루
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,연습했던 손든 인사도 그대로 하면 돼
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,머리스타일 확인하고 한 번 회전 주변


In [5]:
lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])
lyrics.head()

,index,id,title,singer,genres,lyrics,translated_lyrics,sentiment,sentiment_label
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,Ay ay ay ay ay,2,분노
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,거울 속에 내 표정 봐 봐,5,당황
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,느낌 그래서 좋아요. 기다려온 D-하루,0,기쁨
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,연습했던 손든 인사도 그대로 하면 돼,0,기쁨
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,머리스타일 확인하고 한 번 회전 주변,2,분노


### 노래 데이터 최종

In [6]:
# 각 id에 대해 sentiment_label의 빈도를 계산하여, 가장 많이 나온 2개의 감정을 새로운 칼럼으로 추가

def get_top_two_sentiments(sentiments):
    sentiment_counts = sentiments.value_counts()
    # 가장 많은 2개 감정을 반환
    top_two = sentiment_counts.head(2).index.tolist()
    # 2개 미만일 경우 빈 값 처리
    return top_two + [None] * (2 - len(top_two))

# groupby와 agg를 활용해 변환
lyrics_final = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": "first",  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
    "sentiment_label": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment_label에서 두 개의 감정을 각각의 칼럼으로 분리
lyrics_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(lyrics_final['sentiment_label'].tolist(), index=lyrics_final.index)

# 불필요한 sentiment_label 열 삭제
lyrics_final = lyrics_final.drop(columns=["sentiment_label"])

lyrics_final.head()


,id,title,singer,genres,lyrics,translated_lyrics,top_sentiment,second_sentiment
0,418168,희재,성시경,"발라드, 국내영화",햇살은 우릴 위해 내리고,"[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ...",슬픔,기쁨
1,418598,친구라도 될 걸 그랬어,거미 (GUMMY),R&B/Soul,벌써 넌 내가 편하니,"[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는...",불안,슬픔
2,711626,살다가,SG 워너비,발라드,살아도 사는 게 아니래,"[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게...",슬픔,불안
3,1500196,내사람,SG 워너비,R&B/Soul,내 가슴속에 사는 사람 내가 그토록 아끼는 사람,"[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도...",기쁨,불안
4,1854856,라라라,SG 워너비,발라드,그대는 참 아름다워요,"[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감...",기쁨,슬픔


In [10]:
lyrics_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 755 non-null    int64 
 1   title              755 non-null    object
 2   singer             755 non-null    object
 3   genres             755 non-null    object
 4   lyrics             755 non-null    object
 5   translated_lyrics  755 non-null    object
 6   top_sentiment      755 non-null    object
 7   second_sentiment   755 non-null    object
dtypes: int64(1), object(7)
memory usage: 47.3+ KB


## 소설 데이터

In [12]:
novel = pd.read_csv("../0206/results/classified_novel1.csv")
novel.head()

,Column1,file_id,title,paragraph_id,text,labels,sentiment
0,0,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.1,01범보다 무서운 곶감,4.0,불안
1,1,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.2,화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로...,4.0,불안
2,2,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.3,그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 ...,2.0,분노
3,3,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.4,"그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성...",4.0,불안
4,4,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.5,"애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문...",1.0,슬픔


In [13]:
len(novel[novel["file_id"] == "WARW1800000007"])

9543

- 소설의 길이때문에 오류나나.. 해서 100개씩 묶어서 그룹화

In [14]:
# 같은 title을 가진 데이터에서 100개씩 묶어 그룹화
novel["group"] = novel.groupby("title").cumcount() // 100  # 100개 단위 그룹 생성
novel.head()

,Column1,file_id,title,paragraph_id,text,labels,sentiment,group
0,0,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.1,01범보다 무서운 곶감,4.0,불안,0
1,1,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.2,화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로...,4.0,불안,0
2,2,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.3,그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 ...,2.0,분노,0
3,3,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.4,"그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성...",4.0,불안,0
4,4,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.5,"애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문...",1.0,슬픔,0


In [15]:
novel_final = novel.groupby(["file_id", "group"]).agg({
    "title": "first",  # 대표 file_id
    "text": list,  # 100개씩 묶음
    "sentiment": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment에서 두 개의 감정을 각각의 칼럼으로 분리
novel_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(novel_final['sentiment'].tolist(), index=novel_final.index)

# 불필요한 sentiment 칼럼 삭제
novel_final = novel_final.drop(columns=["sentiment", "group", "file_id"])

novel_final.head()

,title,text,top_sentiment,second_sentiment
0,이야기꾼 구연설화,"[01범보다 무서운 곶감, 화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 ...",분노,당황
1,이야기꾼 구연설화,"[<봄꿩은 제 울음에 저 죽는다>, 그 말과 같아서 사램이 잘못 되머넌 하는 말여....",상처,분노
2,이야기꾼 구연설화,"[그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어., “잡수라.”구., 그래 한 때를 몽...",상처,당황
3,이야기꾼 구연설화,"[그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개,, “그러시냐구. 그...",상처,분노
4,이야기꾼 구연설화,"[나와서 인제 그 집 먼지 인저 그, 그, 뭐여 면사무소 있는 디 나와서 인제, 거...",상처,분노


In [16]:
novel_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             8972 non-null   object
 1   text              8972 non-null   object
 2   top_sentiment     8972 non-null   object
 3   second_sentiment  8963 non-null   object
dtypes: object(4)
memory usage: 280.5+ KB


In [17]:
## 두 번째 감정이 없으면 첫 번재랑 같게.
novel_final["second_sentiment"] = novel_final["second_sentiment"].fillna(novel_final["top_sentiment"])


In [19]:
novel_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8972 entries, 0 to 8971
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             8972 non-null   object
 1   text              8972 non-null   object
 2   top_sentiment     8972 non-null   object
 3   second_sentiment  8972 non-null   object
dtypes: object(4)
memory usage: 280.5+ KB


In [20]:
novel_final.to_csv("novel_final.csv")

## KoGPT2

### 사전 테스트

In [21]:
## 이제부턴 이걸로.
#lyrics_final = pd.read_csv("lyrics_final.csv")         이상하게 이렇게 하면 가사 부분 합치는 대서 결과가 달라짐,, 
novel_final = pd.read_csv("novel_final.csv")

In [22]:
# KoGPT2 기본 모델 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [23]:
novel_final["top_sentiment"].unique()

array(['분노', '상처', '당황', '불안', '기쁨', '슬픔'], dtype=object)

In [24]:
# 감정 및 노래 가사 설정
top_sentiment = "슬픔"
second_sentiment = "기쁨"
lyrics = " ".join(lyrics_final["translated_lyrics"][0])
lyrics

'햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니'

In [25]:
# 입력 텍스트 구성
input_text = f"{top_sentiment} | {second_sentiment} | {lyrics}"
input_text

'슬픔 | 기쁨 | 햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니'

In [26]:
# 토큰화
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 문장 생성
gen_ids = model.generate(
    input_ids,
    max_length=500,  # 최대 생성 길이 설정
    repetition_penalty=2.0,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True
)

# 결과 디코딩
generated_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
print(generated_text)

슬픔 | 기쁨 | 햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니 언제나 너를 그리워해요
그대 곁에 있어줘서 고마워요, 그리고 이제부터라도 널 만나고 싶어서 더 이상 미안한 마음 없어요.
사랑하는 나의 모든 것을 다 안고 가겠어요,
그리고 지금껏 당신을 기다리고 있는 당신의 모습을 영원히 잊을 거예요. 오늘 아침신문 보시죠.
먼저 조선일보입니다.
오늘 오전 10시부터 서울 광화문광장에서 '박근혜 퇴진 비상국민행동'이 주최로 열리는 집회가 열렸습니다.
집회 참가자들은 박 대통령의 즉각적인 하야와 국정 정상화를 촉구하고 있습니다.
이들은 "대통령의 사과와 책임자 처벌, 대통령 권한대행 체제 전환" 등을 요구하면서 청문회와 특검 도입 등 검찰 개혁과 국정조사를 요구하고 나섰는데 특히 새누리당 비주류 의원들이 탄핵에 동참할 움직임을 보이고 있다고 전했는데.
새정치민주연합 문재인 대표는 어제 최고위원회의에서 "이번 주말 촛불 민심은 새로운 대한민국을 향한 

- 사전학습된 자료가 뉴스라든가 그런거라서 이렇게 이상하게 나온다..

### Fine-Tuning

https://velog.io/@yeah7598/KoGPT2-%EB%8F%99%ED%99%94-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0

In [27]:
## 이제부턴 이걸로.
novel_final = pd.read_csv("novel_final.csv")

In [28]:
novel_final.head()

,Unnamed: 0,title,text,top_sentiment,second_sentiment
0,0,이야기꾼 구연설화,"['01범보다 무서운 곶감', '화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴...",분노,당황
1,1,이야기꾼 구연설화,"['<봄꿩은 제 울음에 저 죽는다>', '그 말과 같아서 사램이 잘못 되머넌 하는 ...",상처,분노
2,2,이야기꾼 구연설화,"['그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어.', '“잡수라.”구.', '그래 한...",상처,당황
3,3,이야기꾼 구연설화,"['그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개,', '“그러시냐구...",상처,분노
4,4,이야기꾼 구연설화,"['나와서 인제 그 집 먼지 인저 그, 그, 뭐여 면사무소 있는 디 나와서 인제, ...",상처,분노


- 여기 참고해서 이어서 ㄱㄱ

In [29]:
data = novel_final.copy()

In [30]:
data.columns

Index(['Unnamed: 0', 'title', 'text', 'top_sentiment', 'second_sentiment'], dtype='object')

- 감정 : '상처', '불안', '기쁨', '슬픔', '분노', '당황'

In [31]:
from sklearn.preprocessing import LabelEncoder

# 감정 카테고리
emotions = ['상처', '불안', '기쁨', '슬픔', '분노', '당황']

# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions)

LabelEncoder()

In [32]:
# 감정1과 감정2를 숫자로 변환
data['감정1_encoded'] = le.transform(data['top_sentiment'])
data['감정2_encoded'] = le.transform(data['second_sentiment'])

# 수치형 감정1, 감정2와 텍스트 결합
data['input_text'] = data['감정1_encoded'].astype(str) + " | " + data['감정2_encoded'].astype(str) + " | " + data['text']

data['input_text'].head()

0    2 | 1 | ['01범보다 무서운 곶감', '화자를 처음 만나 이야기를 들으러 왔...
1    4 | 2 | ['<봄꿩은 제 울음에 저 죽는다>', '그 말과 같아서 사램이 잘못...
2    4 | 1 | ['그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어.', '“잡수라.”구....
3    4 | 2 | ['그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개,',...
4    4 | 2 | ['나와서 인제 그 집 먼지 인저 그, 그, 뭐여 면사무소 있는 디 ...
Name: input_text, dtype: object

In [33]:
from transformers import PreTrainedTokenizerFast, AutoModelWithLMHead

# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
c:\Users\kimso\anaconda3\envs\ailab\lib\site-packages\transformers\models\auto\modeling_auto.py:1833: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [35]:
# 데이터셋 클래스 정의
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encodes(self, x): 
        # Tokenizing input text with padding and truncation
        encodings = self.tokenizer(x, truncation=True, padding='max_length', max_length=256, return_tensors="pt")
        
        # Extracting input_ids and attention_mask
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        
        print(f"Tokenized: {input_ids.shape} - {input_ids}")
        return input_ids.squeeze(0), attention_mask.squeeze(0)

    def decodes(self, x):
        return self.tokenizer.decode(x, skip_special_tokens=True)


NameError: name 'Transform' is not defined

In [15]:
## 용량? 이슈로 잠시 조정..
data = data[:int(len(data)*0.1)]

In [16]:
# 텍스트 데이터를 리스트 형태로 변환
train = data['input_text'][:int(len(data)*0.9)].tolist()
test = data['input_text'][int(len(data)*0.9):].tolist()

splits = [[0],[1]]

# DataLoader 초기화
tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
batch,seq_len = 4, 64
dls = tls.dataloaders(bs=batch, seq_len=seq_len)

Tokenized: torch.Size([807, 256]) - tensor([[ 9045,   739,   466,  ..., 10912,   387, 12187],
        [ 9130,   739,   466,  ...,  9338,  9236,  7756],
        [ 9130,   739,   466,  ...,  6933, 19202, 18215],
        ...,
        [ 9085,   739,   466,  ...,  8148,  9225,  9939],
        [10595,   739,   466,  ...,  9022, 37994,  9021],
        [ 9085,   739,   466,  ..., 12295, 47043,  9025]])
Tokenized: torch.Size([807, 256]) - tensor([[ 9045,   739,   466,  ..., 10912,   387, 12187],
        [ 9130,   739,   466,  ...,  9338,  9236,  7756],
        [ 9130,   739,   466,  ...,  6933, 19202, 18215],
        ...,
        [ 9085,   739,   466,  ...,  8148,  9225,  9939],
        [10595,   739,   466,  ...,  9022, 37994,  9021],
        [ 9085,   739,   466,  ..., 12295, 47043,  9025]])
Tokenized: torch.Size([90, 256]) - tensor([[ 9143,   739,   466,  ..., 21596,  7312, 10480],
        [10595,   739,   466,  ...,  9205,  9065,  8160],
        [ 9085,   739,   466,  ...,  9394, 44903, 390

In [ ]:
from fastai.text.all import Learner, CrossEntropyLossFlat, Perplexity, Callback

# DataLoader와 모델 확인 후, Learner 초기화
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=Perplexity())

# 모델 fine-tune
learn.fine_tune(1)

epoch,train_loss,valid_loss,perplexity,time


- test

In [ ]:
# 감정1과 감정2를 숫자로 변환
# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions)

lyrics_final['감정1_encoded'] = le.transform(lyrics_final['top_sentiment'])
lyrics_final['감정2_encoded'] = le.transform(lyrics_final['second_sentiment'])


In [ ]:
lyrics_final['감정1_encoded'][3]

In [ ]:
## 숫자 고르기
i = 3

#final = lyrics_final[3]

lyrics = " ".join(lyrics_final["translated_lyrics"][i])

# 'translated_lyrics' 컬럼의 각 리스트 요소를 하나의 문자열로 합침
prompt = lyrics_final['감정1_encoded'][i].astype(str) + " | " + lyrics_final['감정2_encoded'][i].astype(str) + " | " + lyrics
prompt

In [ ]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = learn.model.generate(inp,
                           max_length=256,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 
tokenizer.decode(preds[0].cpu().numpy())